<a href="https://colab.research.google.com/github/hurshd0/DS-Unit-4-Sprint-3-Deep-Learning/blob/module1/module1-rnn-and-lstm/LS_DS_431_RNN_and_LSTM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Check Colab GPU is working

In [2]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

## Which GPU am I using ?

In [3]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 4374675811639964266, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 9319632591351299119
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 13351061907927621600
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11330115994
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9356503702667878830
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

In [0]:
!nvidia-smi

## What about RAM ?

In [4]:
!cat /proc/meminfo 

MemTotal:       26753332 kB
MemFree:        23525580 kB
MemAvailable:   25574760 kB
Buffers:           79172 kB
Cached:          2184748 kB
SwapCached:            0 kB
Active:           959516 kB
Inactive:        1833016 kB
Active(anon):     492920 kB
Inactive(anon):     2376 kB
Active(file):     466596 kB
Inactive(file):  1830640 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:              1668 kB
Writeback:             0 kB
AnonPages:        528728 kB
Mapped:           422716 kB
Shmem:              2944 kB
Slab:             200380 kB
SReclaimable:     149864 kB
SUnreclaim:        50516 kB
KernelStack:        5008 kB
PageTables:         7308 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:    13376664 kB
Committed_AS:    3597260 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
AnonHugePages:         0 kB
ShmemHugePages:  

## What about CPU ?

In [5]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 4
core id		: 0
cpu cores	: 2
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [8]:
import os, sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import tensorflow as tf

from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras import Sequential
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import textwrap

# fix random seed for reproducibility
np.random.seed(45)

Using TensorFlow backend.


### Install `wandb` for colab

In [9]:
in_colab = 'google.colab' in sys.modules

if in_colab:
    !pip install wandb

## Loading and Preprocessing

In [10]:
url = "https://www.gutenberg.org/files/100/100-0.txt"
path_to_file = tf.keras.utils.get_file('shakespeare.txt', url)
raw_text = open(path_to_file, 'r', encoding='utf-8').read()
print(raw_text[:100])

﻿
Project Gutenberg’s The Complete Works of William Shakespeare, by William
Shakespeare

This eBook 


In [11]:
def pre_process(text):
    text = text[900:-21100]
    text = text.replace("\r", "")
    text = " ".join(text.split())
    print (f'Length of text: {len(text)} characters long')
    return text
text = pre_process(raw_text)
text[:100]

Length of text: 5256461 characters long


'Contents THE SONNETS ALL’S WELL THAT ENDS WELL THE TRAGEDY OF ANTONY AND CLEOPATRA AS YOU LIKE IT TH'

In [12]:
# create mapping of unique chars to integers
chars = sorted(set(text))
char2idx = {c:i for i, c in enumerate(chars)}
idx2char = {i:c for i, c in enumerate(chars)}

n_chars = len(text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Unique Characters: ", n_vocab)

Total Characters:  5256461
Total Unique Characters:  99


We can see that the text blob have 500K characters, and that each converted to lowercase with no `\r` gave us 99 unique characters. 

In [13]:
seq_length = 100

X_data= []
y_data = []

for i in range(0, n_chars - seq_length, 1):
    seq_in = text[i:i + seq_length]
    seq_out = text[i + seq_length]
    X_data.append([char2idx[char] for char in seq_in])
    y_data.append(char2idx[seq_out])

n_patterns = len(X_data)
print("Total Patterns: ", n_patterns)

Total Patterns:  5256361


In [14]:
# Reshape X to be [samples, time steps, features]
X = np.reshape(X_data, (n_patterns, seq_length, 1))

# Normalize
X = X / float(n_vocab)

# One hot encode the output variable
y = to_categorical(y_data)

print(f'Shape of X: {X.shape}')
print(f'Shape of y: {y.shape}')

Shape of X: (5256361, 100, 1)
Shape of y: (5256361, 99)


### Create Model

In [15]:
def create_model1(X, y):
    # Define the LSTM model
    model = Sequential()
    model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(y.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    return model


model1 = create_model1(X, y)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


### Add callbacks

In [16]:
import wandb
from wandb.keras import WandbCallback

wandb.init(project="angus")

# define the checkpoint
# Directory where the checkpoints will be saved
checkpoint_directory = './training_checkpoints'

if not os.path.exists(checkpoint_directory ):
    os.makedirs(checkpoint_directory)
    print(f'New directory created at $:->{checkpoint_directory}')
file_name ="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint_filename = os.path.join(checkpoint_directory, file_name)
checkpoint_callback = ModelCheckpoint(checkpoint_filename, monitor='loss', verbose=1, save_best_only=True, mode='min')

callbacks_list = [checkpoint_callback, WandbCallback()]

<IPython.core.display.Javascript object>

wandb: ERROR Not authenticated.  Copy a key from https://app.wandb.ai/authorize


API Key: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


New directory created at $:->./training_checkpoints


### Train/Fit Model

In [0]:
wandb.config.epochs = 20
wandb.config.batch_size = 128
history = model1.fit(X, y, 
                     validation_split=0.33, 
                     batch_size=wandb.config.batch_size, 
                     epochs=wandb.config.epochs, 
                     callbacks=callbacks_list
                    )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 3521761 samples, validate on 1734600 samples
Epoch 1/20
 209920/3521761 [>.............................] - ETA: 1:10:00 - loss: 3.1157

## Plot Loss

In [0]:
plt.plot(history.history['loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN